In [1]:
# Import Statements

%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torchsample
from torchsample import transforms as ts_transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
from tensorboardX import SummaryWriter
from datetime import datetime
import importlib

#from torchsample.transforms import RangeNorm

import functions.fine_tune as ft

plt.ion()   # interactive mode

## 1 - Load Data

Before running the code, the data should be downloaded and foldered in the way that is usable for imagefolder function of the PyTorch. The following code assumes that the main directory for the dataset is 'data_dir' and it includes subdirectories for all of the separate classes.

For details on how to create those folders, pleaserefer to 'dataset/Folder_images.ipynb'

In [2]:
'''
Macros
'''
#uniform_sampler=False
batch_size=32
split=200
random_seed=1
shuffle=True
dataset = 'Amazon'
CV = 0

# Data augmentation and normalization for training 
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.Scale(64),
        transforms.RandomCrop(56),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        #transforms.Scale(64),
        transforms.CenterCrop(56),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


'''Create ImageFolder objects for training and test'''

if dataset == 'Hoarding':
    data_dir = '/media/mtezcan/New Volume/Extreme Picture Finder/all images'
else:
    data_dir = '../images/data_64_64'  
    

if not CV == 0:  
    dset_train = datasets.ImageFolder(data_dir+'/train_val', data_transforms['train'])
    dset_val = datasets.ImageFolder(data_dir+'/train_val', data_transforms['val'])
    
    num_train = len(dset_train)
    indices = list(range(num_train))
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
    splits = (num_train*np.linspace(0,1,CV+1)).astype(int)
    
    val_idx = [indices[splits[k]:splits[k+1]] for k in range(CV)]
    train_idx=[np.setdiff1d(indices,val_idx[k]) for k in range(CV)]
    '''Sampler functions for validation and training'''
    sampler_train = [torch.utils.data.sampler.SubsetRandomSampler(train_idx[k]) for k in range(CV)]
    sampler_val = [torch.utils.data.sampler.SubsetRandomSampler(val_idx[k]) for k in range(CV)]

    '''Define dataset loaders'''
    dset_loaders_arr = [{'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train[k],
                                                        num_workers=12),
                    'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val[k],
                                                        num_workers=12)} for k in range(CV)]
    dset_sizes={'train':int(len(dset_train)*(1-1/CV)),'val':int(len(dset_train)*(1/CV))}
    
    print(dset_sizes)
    print('OR')
    print('Number of training images '+str(len(val_idx)))
    print('Number of validation images '+str(len(train_idx)))

else:
    dset_train = datasets.ImageFolder(data_dir+'/train', data_transforms['train'])
    dset_val = datasets.ImageFolder(data_dir+'/val', data_transforms['val'])

    '''Define dataset loaders'''
    dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,shuffle=True,
                                                        num_workers=12),
                    'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,
                                                        num_workers=12)}

    
    dset_sizes={'train':len(dset_train),'val':len(dset_val)}
    print(dset_sizes)

dset_classes = dset_train.classes
dset_classes_val = dset_val.classes
use_gpu = torch.cuda.is_available()



print(dset_classes)
print(dset_classes_val)

if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')

{'train': 326472, 'val': 35495}
['0', '1', '2', '3', '4', '5']
['0', '1', '2', '3', '4', '5']
GPU is available


In [ ]:
dset_train.ConcatDataset(dset_val)
print(len(dset_train))

In [ ]:
'''
Macros
'''
#uniform_sampler=False
batch_size=64
split=1000
random_seed=1
shuffle=True

# Data augmentation and normalization for training 
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


'''Create ImageFolder objects for training and test'''
data_dir = '../images'
#data_dir = '/home/mtezcan/Documents/amazon/bin_images/0to5'
dset_train = datasets.ImageFolder(data_dir+'/train', data_transforms['train'])
dset_val = datasets.ImageFolder(data_dir+'/val', data_transforms['val'])
'''dset_train = datasets.ImageFolder(data_dir+'/0to5', data_transforms['train'])
dset_val = datasets.ImageFolder(data_dir+'/0to5', data_transforms['val'])'''


'''Validation split'''
num_train = len(dset_train)
indices = list(range(num_train))
val_idx=np.loadtxt('./dataset/validation.txt').astype(np.int)
train_idx=np.setdiff1d(indices,val_idx)

'''if shuffle == True:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]'''

'''Sampler functions for validation and training'''
sampler_train = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(val_idx)

'''Define dataset loaders'''
dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,shuffle=True,
                                                    num_workers=12),
                'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,shuffle=False,
                                                    num_workers=12)}
'''dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train,
                                                    num_workers=12),
                'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val,
                                                    num_workers=12)}'''

dset_sizes={'train':len(dset_train),'val':len(dset_val)}
#dset_sizes={'train':len(dset_train)-1000,'val':1000}
dset_classes = dset_train.classes
dset_classes_val = dset_val.classes
use_gpu = torch.cuda.is_available()

print(dset_sizes)
print('OR')
print('Number of training images '+str(len(val_idx)))
print('Number of validation images '+str(len(train_idx)))

print(dset_classes)
print(dset_classes_val)

if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')

In [ ]:
print(np.max(val_idx))

Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:
inputs, classes = next(iter(dset_loaders['train']))
out = torchvision.utils.make_grid(inputs,nrow=8)
print('Size of the input tensors in one batch after grid is  '+str(out.size()))
plt.figure(figsize=(12,12))
ft.imshow(out, title=[dset_classes[x] for x in classes])

## 2- Log Keeping

This section includes the functions defined for the log keeping. Since CNNs require lots of trials, I found it easy to record the properties of the each trial with their performances in an excel file. I also added tnesorboard summaries for every trial and individual text files for showing the details of the network.

Below code declares the required parameters for the network which will be also used inside log keeping

In [31]:
network='resnet18' #Initial network archtiecture.'loaded'forusing a saved network
networkName='resnet18_real_sgd_multisoft_August29  19:06:27' #Directory for the saved network
optimizer='sgd' #Optimizer function
iter_loc=15 #Number of the first column in the excel file for writing the results.
end_to_end=True #Booolean to decide whether to train the network end-to-end or not.
lr=0.01 #Initial learning rate
momentum=0.9
weight_decay=0.0005
lr_scheduler=ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch=10 #Number of epoch for learning rate decay
pretrained=True 
mse_loss=False #Scalar MSE loss
nclasses=6 #Number of output classes
metric = 'ccr1'
multi_coeff = [1]

if metric == 'ccr1':
    multi_coeff = [1, 1, 1]
if metric == 'mae':
    multi_coeff = (1/8) * [1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1]

'''Multipliers for loss functions'''
single_loss=1.
multi_loss=0.

comment=' ' #Additional comments if any

Let us define the functions for creating a text file and adding networkproperties to an excel file. 

In [4]:
def writeLog(logname):
    '''
    Creates a text file named Network_properties.txt inside runs/'logname'
    '''
    f=open('runs/'+logname+'/Network_properties.txt','w')
    f.write('Batch size: '+str(batch_size)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Random seed: '+str(random_seed)+'\n')
    f.write('Shuffle: '+str(shuffle)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Network: '+network+'\n')
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    f.write('Criterion: '+crt+'\n')
    f.write('Learning rate: '+str(lr)+'\n')
    f.write('Momentum: '+str(momentum)+'\n')
    f.write('Leraning Rate Scheduler: '+str(lr_scheduler)+'\n')
    f.write('Leraning Rate Decay Period: '+str(lr_decay_epoch)+'\n')
    f.write('Network is pretrained: '+str(pretrained)+'\n')
    f.write('Network laoded from: '+networkName+'\n')
    f.write('MSE loss function: '+str(mse_loss)+'\n')
    f.close()

In [26]:
import openpyxl
import time

def writeLog_xlsx(logname='logs.xlsx',iter_loc=15):
    '''
    Adds a line to logs.xlsx with the network properties and outcomes.
    :param iter_loc: First column to record the outcomes.
    '''
    book = openpyxl.load_workbook(logname)
    sheet = book.active
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    if network=='loaded':
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'),networkName,str(split),str(random_seed),str(shuffle),
               optimizer, crt, str(multi_coeff), str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),str(pretrained),
               str(batch_size))
    else:
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'),network,str(split),str(random_seed),str(shuffle),
               optimizer, crt, str(multi_coeff),str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),str(pretrained),
               str(batch_size))
    sheet.append(specs)
    current_row = sheet.max_row
    sheet.cell(row=current_row, column=iter_loc+5).value = comment
    book.save(logname)

## 3 - Training and Validation

In this part we will define the functions for training a CNN with different properties and loss functions.

The following function takes bunch of properties defined in the beginning of Section-2 as input and creates network using those properties.

In [ ]:
model_ft = models.resnet18(pretrained=pretrained)
#model_ft.layer1 = nn.Sequential()#model_ft.layer4
num_ftrs = model_ft.fc.in_features
model_ft.conv1 = nn.Conv2d(3, 128, kernel_size=7, stride=2, padding=3,
                               bias=False)

model_ft.bn1 = nn.BatchNorm2d(128)
model_ft.avgpool = nn.AvgPool2d(2)
print(model_ft)
model_ft.fc = nn.Linear(num_ftrs, nclasses)

In [6]:
def network_loader(comment=comment,
                    network=network,
                    networkName=networkName,
                    optimizer=optimizer,
                    iter_loc=iter_loc,
                    end_to_end=end_to_end,
                    lr=lr,
                    momentum=momentum,
                    weight_decay=weight_decay,
                    lr_scheduler=lr_scheduler,
                    lr_decay_epoch=lr_decay_epoch,
                    pretrained=pretrained,
                    mse_loss=mse_loss,
                    nclasses=nclasses):
    
    '''Load the network from pytorch'''
    if(network=='resnet18'):
        model_ft = models.resnet18(pretrained=pretrained)
        model_ft.conv1 = nn.Conv2d(3, 128, kernel_size=7, stride=2, padding=3,
                               bias=False)

        model_ft.bn1 = nn.BatchNorm2d(128)
        model_ft.layer1 = nn.Sequential()
        model_ft.layer2 = nn.Sequential()
        model_ft.avgpool = nn.AvgPool2d(4)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features #Change the last layer to adapt new classes
        if(mse_loss): #For MSE loss last layer should be a scalar
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet34'):
        model_ft = models.resnet34(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet50'):
        model_ft = models.resnet50(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet101'):
        model_ft = models.resnet101(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='alexnet'):
        model_ft = models.alexnet(pretrained=pretrained)
        num_ftrs = model_ft.classifier[6].out_features
        setattr(model_ft.classifier, '7', nn.ReLU(inplace=True))
        setattr(model_ft.classifier, '8', nn.Dropout())
        setattr(model_ft.classifier, '9', nn.Linear(num_ftrs,nclasses))

    elif(network=='loaded'):#Use the saved network when network='loaded'
        model_ft = torch.load('./saved_models/'+networkName)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, nclasses)
    else:
        raise ValueError('Undefined network '+network)

    if use_gpu:
        model_ft = model_ft.cuda()

    '''Define the optimizer function'''
    if(optimizer=='adam'):
        optimizer_ft = optim.Adam(model_ft.parameters(),lr=lr,weight_decay=weight_decay)
    elif(optimizer=='sgd'):
        if(end_to_end):
            optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=momentum)
        else:
            optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=lr, momentum=momentum,weight_decay=weight_decay)
    return model_ft, optimizer_ft

Now define a simple function to be able to run our training in a single line

In [7]:
def write_epochs(result_log, logname):
    print(len(result_log))

    wb_tr = openpyxl.Workbook()
    ws_tr = wb_tr.active
    wb_val = openpyxl.Workbook()
    ws_val = wb_val.active
    print(logname)

    label_arr_tr = np.zeros((1,1))
    probs_arr_tr = np.zeros((1, nclasses))
    label_arr_val = np.zeros((1,1))
    probs_arr_val = np.zeros((1, nclasses))

    prev_epoch = 0
    for result in result_log:
        epoch = result[1]
        if not epoch == prev_epoch:
            ws_tr.append(['Epoch ' + str(prev_epoch)])
            ws_tr.append(label_arr_tr[1:].reshape(-1).tolist())
            ws_tr.append(np.argmax(probs_arr_tr[1:,:], axis=1).reshape(-1).tolist())
            for probs in probs_arr_tr[1:,:].T.tolist():
                ws_tr.append(probs)
            wb_tr.save('./runs/'+logname + '/train.xlsx')
            ws_val.append(['Epoch ' + str(prev_epoch)])
            ws_val.append(label_arr_val[1:].reshape(-1).tolist())
            ws_val.append(np.argmax(probs_arr_val[1:,:], axis=1).reshape(-1).tolist())
            for probs in probs_arr_val[1:,:].T.tolist():
                ws_val.append(probs)
            wb_val.save('./runs/'+logname + '/val.xlsx')

            del label_arr_tr, label_arr_val, probs_arr_tr, probs_arr_val
            label_arr_tr = np.zeros((1,1))
            probs_arr_tr = np.zeros((1, nclasses))
            label_arr_val = np.zeros((1,1))
            probs_arr_val = np.zeros((1, nclasses))
            prev_epoch = epoch

        label = np.asarray(result[2]).reshape(-1,1)
        scores = np.asarray(result[3])
        exp_scores = np.exp(scores - np.max(scores,axis=1).reshape(-1, 1)*np.ones(nclasses))
        probs = np.round(exp_scores/(np.sum(exp_scores,axis=1).reshape(-1, 1)*np.ones(nclasses)), decimals=2)
        if result[0] == 'train':
            label_arr_tr  =np.append(label_arr_tr, label)
            probs_arr_tr = np.append(probs_arr_tr, probs, axis=0)
        elif result[0] == 'val':
            label_arr_val  =np.append(label_arr_val, label)
            probs_arr_val = np.append(probs_arr_val, probs, axis=0)



    ws_tr.append(['Epoch ' + str(epoch)])
    ws_tr.append(label_arr_tr[1:].reshape(-1).tolist())
    ws_tr.append(np.argmax(probs_arr_tr[1:,:], axis=1).reshape(-1).tolist())
    for probs in probs_arr_tr[1:,:].T.tolist():
        ws_tr.append(probs)
    wb_tr.save('./runs/'+logname + '/train.xlsx')
    ws_val.append(['Epoch ' + str(epoch)])
    ws_val.append(label_arr_val[1:].reshape(-1).tolist())
    ws_val.append(np.argmax(probs_arr_val[1:,:], axis=1).reshape(-1).tolist())
    for probs in probs_arr_val[1:,:].T.tolist():
        ws_val.append(probs)
    wb_val.save('./runs/'+logname + '/val.xlsx')
    label_arr_tr = np.zeros((1,1))
    probs_arr_tr = np.zeros((1, nclasses))
    label_arr_val = np.zeros((1,1))
    probs_arr_val = np.zeros((1, nclasses))
    prev_epoch = epoch
    print('Finito')

In [32]:
importlib.reload(ft)

def run_network():
    print(iter_loc)
    '''
    Cretaes the log files and starts the training
    '''
    model_ft, optimizer_ft = network_loader(comment=comment, #'Tested for three rooms'
                                            network=network,
                                            networkName=networkName,
                                            optimizer=optimizer,
                                            iter_loc=iter_loc,
                                            end_to_end=end_to_end,
                                            lr=lr,
                                            momentum=momentum,
                                            weight_decay=weight_decay,
                                            lr_scheduler=lr_scheduler,
                                            lr_decay_epoch=lr_decay_epoch,
                                            pretrained=pretrained,
                                            mse_loss=mse_loss,
                                            nclasses=nclasses)
    
    '''Name of the trial'''
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    logname=network+'_'+'_'+optimizer+'_'+crt+'_'+datetime.now().strftime('%B%d  %H:%M:%S')
    writer = SummaryWriter('runs/'+logname) #For tensorboard
    writeLog(logname)
    writeLog_xlsx()
    
    '''Start trianing'''
    best_model, last_model, result_log = ft.train_model(model_ft,optimizer_ft, lr_scheduler,dset_loaders,
                            dset_sizes,writer,use_gpu=use_gpu,num_epochs=30,batch_size=batch_size,num_log=250,
                            multi_prob=False,lr_decay_epoch=lr_decay_epoch,init_lr=lr,mse_loss=mse_loss,
                            iter_loc=iter_loc,cross_loss=single_loss,multi_loss=multi_loss,numOut=nclasses,
                            multi_coeff = multi_coeff)
    
    '''Save the models'''
    torch.save(best_model,'./saved_models/'+logname+'_best')
    torch.save(last_model,'./saved_models/'+logname+'_last')
    
    print('Writing results')
    #write_epochs(result_log, logname)
    print('Wrote results')
    '''Free up the memory'''
    del model_ft, result_log
    


## 4 - Trials

The loss function we used in the experiments is the following,

$$loss(\mathbf{y},\hat{y})=(1-\lambda )loss_{single}(\mathbf{y},\hat{y})+\lambda loss_{multi}(\mathbf{y},\hat{y})$$
where $\mathbf{y}$ is the ground truth and $\hat{y}$ is the prediction.

Now lets test our function for different values of $\lambda$.

In [33]:
#importlib.reload(torchvision)
#from torchvision import models
single_loss=0.
multi_loss=1.
run_network()

'''for k in range(len(dset_loaders_arr)):
    dset_loaders = dset_loaders_arr[k]
    run_network()'''

15
Epoch 0/29
----------
LR is set to 0.01
250/10203, acc: 0.2795, CIR-1: 0.7544, RMSE: 1.2960, MAE: 1.0044
500/10203, acc: 0.2889, CIR-1: 0.7724, RMSE: 1.2667, MAE: 0.9772
750/10203, acc: 0.2905, CIR-1: 0.7805, RMSE: 1.2486, MAE: 0.9645
1000/10203, acc: 0.2926, CIR-1: 0.7858, RMSE: 1.2374, MAE: 0.9558
1250/10203, acc: 0.2949, CIR-1: 0.7894, RMSE: 1.2311, MAE: 0.9497
1500/10203, acc: 0.2952, CIR-1: 0.7940, RMSE: 1.2222, MAE: 0.9435
1750/10203, acc: 0.2960, CIR-1: 0.7964, RMSE: 1.2166, MAE: 0.9393
2000/10203, acc: 0.2977, CIR-1: 0.7994, RMSE: 1.2108, MAE: 0.9341
2250/10203, acc: 0.3001, CIR-1: 0.8027, RMSE: 1.2036, MAE: 0.9274
2500/10203, acc: 0.3004, CIR-1: 0.8054, RMSE: 1.1989, MAE: 0.9239
2750/10203, acc: 0.3013, CIR-1: 0.8073, RMSE: 1.1944, MAE: 0.9204
3000/10203, acc: 0.3027, CIR-1: 0.8093, RMSE: 1.1900, MAE: 0.9164
3250/10203, acc: 0.3035, CIR-1: 0.8115, RMSE: 1.1854, MAE: 0.9128
3500/10203, acc: 0.3042, CIR-1: 0.8131, RMSE: 1.1816, MAE: 0.9099
3750/10203, acc: 0.3049, CIR-1: 0.81

ValueError: Invalid column index 326472

In [ ]:
single_loss=0.
multi_loss=1.

for k in range(len(dset_loaders_arr)):
    dset_loaders = dset_loaders_arr[k]
    run_network()
    
single_loss=.1
multi_loss=.9

for k in range(len(dset_loaders_arr)):
    dset_loaders = dset_loaders_arr[k]
    run_network()

In [ ]:
multi_coeff = (1/5) * [1, 2, 3, 4, 5, 4, 3, 2, 1]
run_network()

In [ ]:
multi_coeff = (1/25) * [9, 16, 21, 24, 25, 24, 21, 16, 9]
run_network()

In [ ]:
multi_coeff = (1/5) * np.asarray([1, 2, 3, 4, 5, 4, 3, 2, 1])
print(multi_coeff)
run_network()

multi_coeff = (1/25) * np.asarray([9, 16, 21, 24, 25, 24, 21, 16, 9])
print(multi_coeff)
run_network()

multi_coeff = np.asarray([1])
print(multi_coeff)
run_network()

single_loss=1.
multi_loss=0.
run_network()

[ 0.2  0.4  0.6  0.8  1.   0.8  0.6  0.4  0.2]
15
Epoch 0/29
----------
LR is set to 0.01
250/10203, acc: 0.2814, CIR-1: 0.7411, RMSE: 1.3170, MAE: 1.0190
500/10203, acc: 0.2923, CIR-1: 0.7560, RMSE: 1.2802, MAE: 0.9879
750/10203, acc: 0.2997, CIR-1: 0.7697, RMSE: 1.2543, MAE: 0.9643
1000/10203, acc: 0.3040, CIR-1: 0.7758, RMSE: 1.2416, MAE: 0.9526
1250/10203, acc: 0.3059, CIR-1: 0.7809, RMSE: 1.2309, MAE: 0.9440
1500/10203, acc: 0.3084, CIR-1: 0.7850, RMSE: 1.2218, MAE: 0.9361
1750/10203, acc: 0.3087, CIR-1: 0.7887, RMSE: 1.2142, MAE: 0.9309
2000/10203, acc: 0.3113, CIR-1: 0.7925, RMSE: 1.2075, MAE: 0.9242
2250/10203, acc: 0.3129, CIR-1: 0.7953, RMSE: 1.2016, MAE: 0.9189
2500/10203, acc: 0.3141, CIR-1: 0.7972, RMSE: 1.1963, MAE: 0.9148
2750/10203, acc: 0.3154, CIR-1: 0.7990, RMSE: 1.1924, MAE: 0.9113
3000/10203, acc: 0.3173, CIR-1: 0.8009, RMSE: 1.1883, MAE: 0.9071
3250/10203, acc: 0.3183, CIR-1: 0.8024, RMSE: 1.1846, MAE: 0.9040
3500/10203, acc: 0.3195, CIR-1: 0.8034, RMSE: 1.1822, M

In [ ]:
print(result_log)

In [ ]:
mse_loss=False
for lmbda in [1., 1., 1., 1., 1.]:
    single_loss= round(1.-lmbda, 1)
    multi_loss = lmbda
    print('Single loss = '+str(single_loss)+', Multi loss = '+str(multi_loss))
    run_network()